In [2]:
import pandas as pd

In [4]:
# Load phases dataframe
parse_dates = ["start", "end", "travel_date", "latest_res_dt"]
df = pd.read_csv("model_data_weather.csv", parse_dates=parse_dates)
df

,start,start_loc,count,end,end_loc,capacity,line,travel_date,latest_res_dt,date,leisure_biking:idx,t_2m:C,precip_24h:mm
0,2021-03-21 06:34:30,ZUE,1.0,2021-03-21 07:28:00,BS,15.0,IC 3,2021-03-21,2021-03-21 06:34:30,2021-03-21,0.238462,1.938462,0.218462
1,2021-03-21 06:38:30,ZUE,1.0,2021-03-21 07:52:00,CH,9.0,IC 3,2021-03-21,2021-03-21 06:38:30,2021-03-21,0.238462,1.938462,0.218462
2,2021-03-21 09:38:30,ZUE,2.0,2021-03-21 10:52:00,CH,24.0,IC 3,2021-03-21,2021-03-21 09:38:30,2021-03-21,0.238462,1.938462,0.218462
3,2021-03-21 09:08:30,CH,2.0,2021-03-21 09:18:42,LQ,9.0,IC 3,2021-03-21,2021-03-21 09:08:30,2021-03-21,0.238462,1.938462,0.218462
4,2021-03-21 09:18:42,LQ,3.0,2021-03-21 10:22:18,ZUE,9.0,IC 3,2021-03-21,2021-03-21 09:18:42,2021-03-21,0.238462,1.938462,0.218462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115769,2021-03-30 06:59:30,W,1.0,2021-03-30 07:32:00,SG,12.0,IC 5,2021-03-30,2021-03-30 06:59:30,2021-03-30,0.488462,13.753846,0.002308
115770,2021-03-30 19:49:30,BI,1.0,2021-03-30 20:53:30,BS,6.0,IC 51,2021-03-30,2021-03-30 19:49:30,2021-03-30,0.488462,13.753846,0.002308
115771,2021-03-30 05:43:30,BI,2.0,2021-03-30 06:01:54,SO,12.0,IC 5,2021-03-30,2021-03-30 05:43:30,2021-03-30,0.488462,13.753846,0.002308
115772,2021-03-30 06:01:54,SO,3.0,2021-03-30 06:56:00,ZUE,12.0,IC 5,2021-03-30,2021-03-30 06:01:54,2021-03-30,0.488462,13.753846,0.002308


In [7]:
# train station coordinates data
df_coordinates = pd.read_csv('../data_preprocessed/dienststellen.csv')
df_coordinates = df_coordinates[["abk_bahnhof", "lat", "lon"]]
df_coordinates = df_coordinates[df_coordinates['abk_bahnhof'].notna()]
display(df_coordinates.tail(1))

/Users/oli/miniforge3/envs/tensorflow/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,abk_bahnhof,lat,lon
47113,BRIA,47.260083,7.942674


In [8]:
# full join for start train station 
df = pd.merge(df, df_coordinates, left_on='start_loc', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_from", "lon": "lon_from"})

# full join for destination
df = pd.merge(df, df_coordinates, left_on='end_loc', right_on='abk_bahnhof')
df = df.drop(columns=["abk_bahnhof", "start_loc", "end_loc"]).rename(columns={"lat": "lat_to", "lon": "lon_to"})
df

,start,count,end,capacity,line,travel_date,latest_res_dt,date,leisure_biking:idx,t_2m:C,precip_24h:mm,lat_from,lon_from,lat_to,lon_to
0,2021-03-21 06:34:30,1.0,2021-03-21 07:28:00,15.0,IC 3,2021-03-21,2021-03-21 06:34:30,2021-03-21,0.238462,1.938462,0.218462,47.378177,8.540212,47.547412,7.589563
1,2021-03-26 06:34:30,1.0,2021-03-26 07:28:00,9.0,IC 3,2021-03-26,2021-03-26 06:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563
2,2021-03-26 10:34:30,1.0,2021-03-26 11:28:00,9.0,IC 3,2021-03-26,2021-03-26 10:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563
3,2021-03-26 14:34:30,1.0,2021-03-26 15:28:00,9.0,IC 3,2021-03-26,2021-03-26 14:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563
4,2021-03-26 16:34:30,1.0,2021-03-26 17:28:00,13.0,IC 3,2021-03-26,2021-03-26 16:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115769,2021-09-24 21:32:06,3.0,2021-09-24 22:32:12,12.0,IC 5,2021-09-24,2021-09-24 21:32:06,2021-09-24,0.923077,17.561538,0.000000,46.996727,6.935702,46.384419,6.235966
115770,2021-09-04 20:58:30,1.0,2021-09-04 22:32:12,6.0,IC 5,2021-09-04,2021-09-04 20:58:30,2021-09-04,0.450000,18.523077,2.086923,47.204192,7.542689,46.384419,6.235966
115771,2021-08-28 21:52:00,3.0,2021-08-28 22:32:12,6.0,IC 5,2021-08-28,2021-08-28 21:52:00,2021-08-28,0.626923,14.538462,0.148462,46.781545,6.640944,46.384419,6.235966
115772,2021-09-24 22:32:12,2.0,2021-09-24 22:32:12,12.0,IC 5,2021-09-24,2021-09-24 22:32:12,2021-09-24,0.923077,17.561538,0.000000,46.384419,6.235966,46.384419,6.235966


## Feature Engineering

In [9]:
df_book_in_advance = pd.to_timedelta(df["start"] - df["latest_res_dt"], unit="m").dt.total_seconds()//60
df_book_in_advance.drop(df_book_in_advance[df_book_in_advance < 0].index, inplace=True)
df_book_in_advance.describe()

count    115749.000000
mean         69.137910
std        1142.966972
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max       88444.000000
dtype: float64

In [10]:
df["book_in_advance"] = df_book_in_advance
df

,start,count,end,capacity,line,travel_date,latest_res_dt,date,leisure_biking:idx,t_2m:C,precip_24h:mm,lat_from,lon_from,lat_to,lon_to,book_in_advance
0,2021-03-21 06:34:30,1.0,2021-03-21 07:28:00,15.0,IC 3,2021-03-21,2021-03-21 06:34:30,2021-03-21,0.238462,1.938462,0.218462,47.378177,8.540212,47.547412,7.589563,0.0
1,2021-03-26 06:34:30,1.0,2021-03-26 07:28:00,9.0,IC 3,2021-03-26,2021-03-26 06:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0
2,2021-03-26 10:34:30,1.0,2021-03-26 11:28:00,9.0,IC 3,2021-03-26,2021-03-26 10:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0
3,2021-03-26 14:34:30,1.0,2021-03-26 15:28:00,9.0,IC 3,2021-03-26,2021-03-26 14:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0
4,2021-03-26 16:34:30,1.0,2021-03-26 17:28:00,13.0,IC 3,2021-03-26,2021-03-26 16:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115769,2021-09-24 21:32:06,3.0,2021-09-24 22:32:12,12.0,IC 5,2021-09-24,2021-09-24 21:32:06,2021-09-24,0.923077,17.561538,0.000000,46.996727,6.935702,46.384419,6.235966,0.0
115770,2021-09-04 20:58:30,1.0,2021-09-04 22:32:12,6.0,IC 5,2021-09-04,2021-09-04 20:58:30,2021-09-04,0.450000,18.523077,2.086923,47.204192,7.542689,46.384419,6.235966,0.0
115771,2021-08-28 21:52:00,3.0,2021-08-28 22:32:12,6.0,IC 5,2021-08-28,2021-08-28 21:52:00,2021-08-28,0.626923,14.538462,0.148462,46.781545,6.640944,46.384419,6.235966,0.0
115772,2021-09-24 22:32:12,2.0,2021-09-24 22:32:12,12.0,IC 5,2021-09-24,2021-09-24 22:32:12,2021-09-24,0.923077,17.561538,0.000000,46.384419,6.235966,46.384419,6.235966,0.0


### Replace start and end time with better representation

In [11]:
# add day of the week
df["day_of_week"] = df["travel_date"].dt.day_of_week
# add week number
df["week"] = df["travel_date"].dt.isocalendar().week
# add year
df["year"] = df["travel_date"].dt.year
df_data = df.drop(["count", "travel_date"], axis=1)
df_data

,start,end,capacity,line,latest_res_dt,date,leisure_biking:idx,t_2m:C,precip_24h:mm,lat_from,lon_from,lat_to,lon_to,book_in_advance,day_of_week,week,year
0,2021-03-21 06:34:30,2021-03-21 07:28:00,15.0,IC 3,2021-03-21 06:34:30,2021-03-21,0.238462,1.938462,0.218462,47.378177,8.540212,47.547412,7.589563,0.0,6,11,2021
1,2021-03-26 06:34:30,2021-03-26 07:28:00,9.0,IC 3,2021-03-26 06:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0,4,12,2021
2,2021-03-26 10:34:30,2021-03-26 11:28:00,9.0,IC 3,2021-03-26 10:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0,4,12,2021
3,2021-03-26 14:34:30,2021-03-26 15:28:00,9.0,IC 3,2021-03-26 14:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0,4,12,2021
4,2021-03-26 16:34:30,2021-03-26 17:28:00,13.0,IC 3,2021-03-26 16:34:30,2021-03-26,0.369231,8.023077,0.005385,47.378177,8.540212,47.547412,7.589563,0.0,4,12,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115769,2021-09-24 21:32:06,2021-09-24 22:32:12,12.0,IC 5,2021-09-24 21:32:06,2021-09-24,0.923077,17.561538,0.000000,46.996727,6.935702,46.384419,6.235966,0.0,4,38,2021
115770,2021-09-04 20:58:30,2021-09-04 22:32:12,6.0,IC 5,2021-09-04 20:58:30,2021-09-04,0.450000,18.523077,2.086923,47.204192,7.542689,46.384419,6.235966,0.0,5,35,2021
115771,2021-08-28 21:52:00,2021-08-28 22:32:12,6.0,IC 5,2021-08-28 21:52:00,2021-08-28,0.626923,14.538462,0.148462,46.781545,6.640944,46.384419,6.235966,0.0,5,34,2021
115772,2021-09-24 22:32:12,2021-09-24 22:32:12,12.0,IC 5,2021-09-24 22:32:12,2021-09-24,0.923077,17.561538,0.000000,46.384419,6.235966,46.384419,6.235966,0.0,4,38,2021


### Remove missing data

In [12]:
# check for missing data 
df_data.isna().sum()

start                  0
end                    0
capacity               0
line                   0
latest_res_dt          0
date                   0
leisure_biking:idx     0
t_2m:C                 0
precip_24h:mm          0
lat_from               0
lon_from               0
lat_to                 0
lon_to                 0
book_in_advance       25
day_of_week            0
week                   0
year                   0
dtype: int64

In [13]:
print(df_data.shape)
df_data = df_data.dropna()
print(df_data.shape)

(115774, 17)
(115749, 17)


In [14]:
df_data.to_csv("../data_preprocessed/df_data_with_weather.csv", index=False)